<a href="https://colab.research.google.com/github/RautRitesh/langgraph/blob/main/financial_analyst_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
pip install yfinance tavily-python langchain langchain-community langgraph langchain-groq langchain-tavily

##Step1 importing the Required Library🤓🤓

In [10]:
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_tavily import TavilySearch
import yfinance as yf
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import MessagesState, StateGraph, START, END
import requests
from langchain_core.tools import tool

In [3]:
from google.colab import userdata
tavily_api_key=userdata.get('Tavily_api_key')
api_key=userdata.get('groq_api_key')

In [5]:
llm=ChatGroq(model="openai/gpt-oss-20b",api_key=api_key)

##Step2: Now Time for setting up tools🪚🪚🔨⚒️⚒️

In [17]:
@tool
def lookup_ticker(company_name: str):
    """
    Search for the correct stock ticker symbol given a company name.
    Use this FIRST if the user provides a name like "Apple", "Tesla", or "Diyo.ai".
    """
    url = "https://query2.finance.yahoo.com/v1/finance/search"
    headers = {'User-Agent': 'Mozilla/5.0'}
    params = {"q": company_name, "quotes_count": 1, "country": "United States"}

    try:
        res = requests.get(url, params=params, headers=headers)
        data = res.json()
        if "quotes" in data and len(data["quotes"]) > 0:
            symbol = data["quotes"][0]["symbol"]
            return f"The ticker for {company_name} is {symbol}."
        return f"No ticker found for {company_name}."
    except Exception as e:
        return f"Error looking up ticker: {e}"

@tool
def get_stock_price(ticker: str):
    """
    Get the current stock price for a known ticker symbol (e.g., AAPL, TSLA).
    Do NOT use this tool with a company name; find the ticker first.
    """
    try:
        stock = yf.Ticker(ticker)
        # Fast retrieval of the last closing price
        hist = stock.history(period="2d")
        if hist.empty:
            return f"Could not fetch price data for {ticker}. Is the ticker correct?"
        price = hist['Close'].iloc[-1]
        return f"The current price of {ticker.upper()} is ${price:.2f}."
    except Exception as e:
        return f"Error fetching price for {ticker}: {e}"

In [18]:
ticker=lookup_ticker.invoke("Apple")
print(ticker)
stock_price=get_stock_price.invoke("APPL")

ERROR:yfinance:$APPL: possibly delisted; no price data found  (period=2d)


The ticker for Apple is AAPL.


In [11]:
tavily_search=TavilySearch(max_results=5,search_depth="advanced",tavily_api_key=tavily_api_key)

In [13]:
tools=[lookup_ticker,get_stock_price,tavily_search]